In [ ]:
# Core data manipulation and analysis
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Time series analysis and forecasting
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Model evaluation metrics
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from math import sqrt

# Set visualization styles
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ All libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")


In [ ]:
def generate_campaign_data(days=180, start_date='2024-01-01'):
    """
    Generate synthetic campaign KPI data with realistic patterns and correlations.
    
    Parameters:
    - days: Number of days to generate data for
    - start_date: Starting date for the campaign
    
    Returns:
    - DataFrame with daily campaign KPIs
    """
    
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Create date range
    dates = pd.date_range(start=start_date, periods=days, freq='D')
    
    # Initialize base metrics with trend and seasonality
    base_trend = np.linspace(1000, 1200, days)  # Gradual increase in activity
    
    # Weekly seasonality (higher activity on weekdays)
    weekly_pattern = np.sin(2 * np.pi * np.arange(days) / 7) * 0.15 + 1
    
    # Monthly seasonality (business cycles)
    monthly_pattern = np.sin(2 * np.pi * np.arange(days) / 30) * 0.1 + 1
    
    # Random noise
    noise = np.random.normal(0, 0.08, days)
    
    # Generate daily spend with seasonality and trend
    daily_spend = base_trend * weekly_pattern * monthly_pattern * (1 + noise)
    daily_spend = np.maximum(daily_spend, 500)  # Minimum spend threshold
    
    # Generate impressions (correlated with spend, but with platform efficiency variations)
    cpm_base = 2.5  # Base CPM
    cpm_variation = np.random.normal(0, 0.3, days)
    impressions = (daily_spend / (cpm_base + cpm_variation)) * 1000
    impressions = np.maximum(impressions, 10000)  # Minimum impressions
    
    # Generate clicks (CTR varies with day of week and campaign optimization)
    base_ctr = 0.025  # 2.5% base CTR
    ctr_optimization = np.linspace(0, 0.008, days)  # CTR improves over time
    day_of_week_effect = np.array([0.8, 1.0, 1.1, 1.2, 1.15, 0.9, 0.7])  # Mon-Sun multipliers
    dow_multiplier = np.array([day_of_week_effect[date.weekday()] for date in dates])
    
    ctr = (base_ctr + ctr_optimization) * dow_multiplier * (1 + np.random.normal(0, 0.1, days))
    clicks = impressions * ctr
    
    # Generate conversions (CVR improves with campaign optimization)
    base_cvr = 0.08  # 8% base conversion rate
    cvr_optimization = np.linspace(0, 0.02, days)  # CVR improves over time
    cvr = (base_cvr + cvr_optimization) * (1 + np.random.normal(0, 0.15, days))
    conversions = clicks * cvr
    
    # Calculate derived metrics
    cpc = daily_spend / clicks
    cpa = daily_spend / conversions
    
    # Generate revenue (assumes average order value with some variation)
    avg_order_value = np.random.normal(150, 25, days)
    avg_order_value = np.maximum(avg_order_value, 50)  # Minimum AOV
    revenue = conversions * avg_order_value
    
    # Calculate ROAS
    roas = revenue / daily_spend
    
    # Create DataFrame
    campaign_data = pd.DataFrame({
        'date': dates,
        'spend': daily_spend,
        'impressions': impressions,
        'clicks': clicks,
        'conversions': conversions,
        'cpc': cpc,
        'cpa': cpa,
        'revenue': revenue,
        'roas': roas,
        'ctr': ctr * 100,  # Convert to percentage
        'cvr': cvr * 100   # Convert to percentage
    })
    
    # Round numerical values for realism
    campaign_data['spend'] = campaign_data['spend'].round(2)
    campaign_data['impressions'] = campaign_data['impressions'].round(0).astype(int)
    campaign_data['clicks'] = campaign_data['clicks'].round(0).astype(int)
    campaign_data['conversions'] = campaign_data['conversions'].round(0).astype(int)
    campaign_data['revenue'] = campaign_data['revenue'].round(2)
    campaign_data['cpc'] = campaign_data['cpc'].round(3)
    campaign_data['cpa'] = campaign_data['cpa'].round(2)
    campaign_data['roas'] = campaign_data['roas'].round(2)
    campaign_data['ctr'] = campaign_data['ctr'].round(3)
    campaign_data['cvr'] = campaign_data['cvr'].round(3)
    
    return campaign_data

# Generate the campaign data
print("🔄 Generating synthetic campaign data...")
df = generate_campaign_data(days=180, start_date='2024-01-01')

# Display basic information
print(f"✅ Generated {len(df)} days of campaign data")
print(f"📅 Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"💰 Total spend: ${df['spend'].sum():,.2f}")
print(f"📊 Total conversions: {df['conversions'].sum():,.0f}")
print(f"📈 Average ROAS: {df['roas'].mean():.2f}")

# Display first few rows
print("\n📋 Sample of generated data:")
df.head()


In [ ]:
# Summary statistics
print("📈 Campaign KPI Summary Statistics")
print("=" * 50)
summary_stats = df[['spend', 'impressions', 'clicks', 'conversions', 'cpc', 'cpa', 'roas', 'ctr', 'cvr']].describe()
print(summary_stats.round(3))

# Correlation matrix
print("\n🔗 Correlation Matrix")
print("=" * 30)
correlation_matrix = df[['spend', 'impressions', 'clicks', 'conversions', 'cpc', 'cpa', 'roas', 'ctr', 'cvr']].corr()
print(correlation_matrix.round(3))


In [ ]:
# Create comprehensive visualizations
fig, axes = plt.subplots(3, 2, figsize=(15, 18))
fig.suptitle('Campaign KPI Trends Over Time', fontsize=16, fontweight='bold')

# 1. Daily Spend Trend
axes[0, 0].plot(df['date'], df['spend'], color='#1f77b4', linewidth=2)
axes[0, 0].set_title('Daily Spend Trend', fontweight='bold')
axes[0, 0].set_ylabel('Spend ($)')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Impressions and Clicks
ax2 = axes[0, 1]
ax2.plot(df['date'], df['impressions'], color='#ff7f0e', label='Impressions', linewidth=2)
ax2_twin = ax2.twinx()
ax2_twin.plot(df['date'], df['clicks'], color='#2ca02c', label='Clicks', linewidth=2)
ax2.set_title('Impressions vs Clicks', fontweight='bold')
ax2.set_ylabel('Impressions', color='#ff7f0e')
ax2_twin.set_ylabel('Clicks', color='#2ca02c')
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

# 3. Conversions Trend
axes[1, 0].plot(df['date'], df['conversions'], color='#d62728', linewidth=2)
axes[1, 0].set_title('Daily Conversions', fontweight='bold')
axes[1, 0].set_ylabel('Conversions')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].tick_params(axis='x', rotation=45)

# 4. CPA Trend
axes[1, 1].plot(df['date'], df['cpa'], color='#9467bd', linewidth=2)
axes[1, 1].set_title('Cost Per Acquisition (CPA)', fontweight='bold')
axes[1, 1].set_ylabel('CPA ($)')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

# 5. ROAS Trend
axes[2, 0].plot(df['date'], df['roas'], color='#8c564b', linewidth=2)
axes[2, 0].set_title('Return on Ad Spend (ROAS)', fontweight='bold')
axes[2, 0].set_ylabel('ROAS')
axes[2, 0].axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Break-even')
axes[2, 0].legend()
axes[2, 0].grid(True, alpha=0.3)
axes[2, 0].tick_params(axis='x', rotation=45)

# 6. CTR and CVR Trends
axes[2, 1].plot(df['date'], df['ctr'], color='#e377c2', label='CTR (%)', linewidth=2)
axes[2, 1].plot(df['date'], df['cvr'], color='#7f7f7f', label='CVR (%)', linewidth=2)
axes[2, 1].set_title('Click-Through Rate & Conversion Rate', fontweight='bold')
axes[2, 1].set_ylabel('Rate (%)')
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3)
axes[2, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Weekly pattern analysis
df['day_of_week'] = df['date'].dt.day_name()
df['week_number'] = df['date'].dt.isocalendar().week

# Day of week analysis
plt.figure(figsize=(12, 8))

# Subplot 1: Average metrics by day of week
plt.subplot(2, 2, 1)
daily_avg = df.groupby('day_of_week')[['spend', 'conversions', 'roas']].mean()
daily_avg = daily_avg.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
daily_avg['spend'].plot(kind='bar', color='skyblue', alpha=0.7)
plt.title('Average Daily Spend by Day of Week')
plt.ylabel('Spend ($)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
daily_avg['conversions'].plot(kind='bar', color='lightcoral', alpha=0.7)
plt.title('Average Conversions by Day of Week')
plt.ylabel('Conversions')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
daily_avg['roas'].plot(kind='bar', color='lightgreen', alpha=0.7)
plt.title('Average ROAS by Day of Week')
plt.ylabel('ROAS')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Correlation heatmap
plt.subplot(2, 2, 4)
metrics_for_corr = ['spend', 'impressions', 'clicks', 'conversions', 'cpa', 'roas']
corr_matrix = df[metrics_for_corr].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, square=True, cbar_kws={'shrink': 0.8})
plt.title('KPI Correlation Matrix')

plt.tight_layout()
plt.show()

print("📊 Key Observations:")
print(f"• Best performing day: {daily_avg['roas'].idxmax()} (ROAS: {daily_avg['roas'].max():.2f})")
print(f"• Highest spend day: {daily_avg['spend'].idxmax()} (${daily_avg['spend'].max():.2f})")
print(f"• Most conversions: {daily_avg['conversions'].idxmax()} ({daily_avg['conversions'].max():.0f} conversions)")
print(f"• Strongest correlation: {corr_matrix.unstack().drop_duplicates().sort_values(ascending=False).iloc[1]:.3f}")


In [ ]:
# Prepare data for time series modeling
df_ts = df.set_index('date')

# Key metrics to forecast
metrics_to_forecast = ['spend', 'conversions', 'cpa', 'roas']

# Split data for training and testing (80/20 split)
train_size = int(len(df_ts) * 0.8)
train_data = df_ts[:train_size]
test_data = df_ts[train_size:]

print(f"📊 Data Split:")
print(f"• Training period: {train_data.index[0].date()} to {train_data.index[-1].date()} ({len(train_data)} days)")
print(f"• Testing period: {test_data.index[0].date()} to {test_data.index[-1].date()} ({len(test_data)} days)")

# Function to check stationarity
def check_stationarity(timeseries, title):
    """
    Check if a time series is stationary using Augmented Dickey-Fuller test
    """
    print(f"🔍 Stationarity Test Results for {title}:")
    print("-" * 40)
    
    # Perform Augmented Dickey-Fuller test
    result = adfuller(timeseries.dropna())
    
    print(f'ADF Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value:.3f}')
    
    if result[1] <= 0.05:
        print("✅ Series is stationary (reject null hypothesis)")
    else:
        print("❌ Series is non-stationary (fail to reject null hypothesis)")
    print()

# Check stationarity for key metrics
for metric in metrics_to_forecast:
    check_stationarity(train_data[metric], metric.upper())


In [ ]:
# Model building and forecasting
forecast_results = {}
model_performance = {}

def build_arima_model(data, metric_name, order=(1,1,1)):
    """
    Build and train ARIMA model
    """
    try:
        model = ARIMA(data, order=order)
        fitted_model = model.fit()
        return fitted_model
    except Exception as e:
        print(f"⚠️ ARIMA model failed for {metric_name}: {str(e)}")
        return None

def build_ets_model(data, metric_name, trend='add', seasonal='add', seasonal_periods=7):
    """
    Build and train ETS model
    """
    try:
        model = ETSModel(data, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods)
        fitted_model = model.fit()
        return fitted_model
    except Exception as e:
        print(f"⚠️ ETS model failed for {metric_name}: {str(e)}")
        return None

def calculate_metrics(actual, predicted):
    """
    Calculate forecast accuracy metrics
    """
    mape = mean_absolute_percentage_error(actual, predicted) * 100
    rmse = sqrt(mean_squared_error(actual, predicted))
    mae = np.mean(np.abs(actual - predicted))
    
    return {'MAPE': mape, 'RMSE': rmse, 'MAE': mae}

# Forecast horizon (number of days to predict)
forecast_horizon = len(test_data)

print("🔄 Building forecasting models...")
print("=" * 50)

for metric in metrics_to_forecast:
    print(f"\n📈 Forecasting {metric.upper()}:")
    print("-" * 30)
    
    # Prepare data
    train_series = train_data[metric]
    test_series = test_data[metric]
    
    # Build ARIMA model
    print("Building ARIMA model...")
    arima_model = build_arima_model(train_series, metric)
    
    # Build ETS model
    print("Building ETS model...")
    ets_model = build_ets_model(train_series, metric)
    
    # Store models and make forecasts
    forecast_results[metric] = {
        'actual': test_series,
        'arima_model': arima_model,
        'ets_model': ets_model
    }
    
    if arima_model is not None:
        arima_forecast = arima_model.forecast(steps=forecast_horizon)
        forecast_results[metric]['arima_forecast'] = arima_forecast
        
        # Calculate performance metrics
        arima_metrics = calculate_metrics(test_series.values, arima_forecast)
        model_performance[f'{metric}_arima'] = arima_metrics
        print(f"ARIMA - MAPE: {arima_metrics['MAPE']:.2f}%, RMSE: {arima_metrics['RMSE']:.2f}")
    
    if ets_model is not None:
        ets_forecast = ets_model.forecast(steps=forecast_horizon)
        forecast_results[metric]['ets_forecast'] = ets_forecast
        
        # Calculate performance metrics
        ets_metrics = calculate_metrics(test_series.values, ets_forecast)
        model_performance[f'{metric}_ets'] = ets_metrics
        print(f"ETS - MAPE: {ets_metrics['MAPE']:.2f}%, RMSE: {ets_metrics['RMSE']:.2f}")

print("\n✅ Model building completed!")


In [ ]:
# Visualize forecast results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Forecast Results: ARIMA vs ETS Models', fontsize=16, fontweight='bold')

for i, metric in enumerate(metrics_to_forecast):
    row = i // 2
    col = i % 2
    
    ax = axes[row, col]
    
    # Plot historical data
    train_data[metric].plot(ax=ax, label='Training Data', color='blue', alpha=0.7, linewidth=1.5)
    test_data[metric].plot(ax=ax, label='Actual Test Data', color='black', linewidth=2)
    
    # Plot forecasts if available
    if 'arima_forecast' in forecast_results[metric]:
        ax.plot(test_data.index, forecast_results[metric]['arima_forecast'], 
                label='ARIMA Forecast', color='red', linestyle='--', linewidth=2)
    
    if 'ets_forecast' in forecast_results[metric]:
        ax.plot(test_data.index, forecast_results[metric]['ets_forecast'], 
                label='ETS Forecast', color='green', linestyle=':', linewidth=2)
    
    ax.set_title(f'{metric.upper()} Forecasting Results', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Performance comparison table
print("📊 Model Performance Comparison")
print("=" * 60)

performance_df = pd.DataFrame(model_performance).T
performance_df.index = [idx.replace('_', ' - ').upper() for idx in performance_df.index]

print(performance_df.round(3))

# Identify best models for each metric
print("\n🏆 Best Performing Models:")
print("-" * 40)

for metric in metrics_to_forecast:
    arima_key = f'{metric}_arima'
    ets_key = f'{metric}_ets'
    
    if arima_key in model_performance and ets_key in model_performance:
        arima_mape = model_performance[arima_key]['MAPE']
        ets_mape = model_performance[ets_key]['MAPE']
        
        best_model = 'ARIMA' if arima_mape < ets_mape else 'ETS'
        best_mape = min(arima_mape, ets_mape)
        
        print(f"• {metric.upper()}: {best_model} (MAPE: {best_mape:.2f}%)")
    elif arima_key in model_performance:
        print(f"• {metric.upper()}: ARIMA (MAPE: {model_performance[arima_key]['MAPE']:.2f}%)")
    elif ets_key in model_performance:
        print(f"• {metric.upper()}: ETS (MAPE: {model_performance[ets_key]['MAPE']:.2f}%)")


In [ ]:
# Budget reallocation scenarios based on forecasted performance
def simulate_budget_scenarios(current_spend, forecast_cpa, forecast_roas, forecast_conversions):
    """
    Simulate different budget allocation scenarios
    """
    scenarios = {}
    
    # Scenario 1: Current baseline
    scenarios['Current'] = {
        'daily_spend': current_spend.mean(),
        'total_spend': current_spend.sum(),
        'avg_cpa': forecast_cpa.mean(),
        'avg_roas': forecast_roas.mean(),
        'total_conversions': forecast_conversions.sum(),
        'efficiency_score': forecast_roas.mean() / forecast_cpa.mean()
    }
    
    # Scenario 2: Optimize for ROAS (reduce spend on low-ROAS days)
    roas_threshold = forecast_roas.quantile(0.3)  # Bottom 30% ROAS days
    optimized_spend = current_spend.copy()
    optimized_spend[forecast_roas < roas_threshold] *= 0.7  # Reduce spend by 30%
    
    scenarios['ROAS Optimized'] = {
        'daily_spend': optimized_spend.mean(),
        'total_spend': optimized_spend.sum(),
        'avg_cpa': forecast_cpa.mean() * 0.85,  # Estimated CPA improvement
        'avg_roas': forecast_roas.mean() * 1.15,  # Estimated ROAS improvement
        'total_conversions': forecast_conversions.sum() * 0.95,  # Slight conversion decrease
        'efficiency_score': (forecast_roas.mean() * 1.15) / (forecast_cpa.mean() * 0.85)
    }
    
    # Scenario 3: Aggressive cost reduction (25% overall spend cut)
    cost_reduction_spend = current_spend * 0.75
    
    scenarios['Cost Reduction'] = {
        'daily_spend': cost_reduction_spend.mean(),
        'total_spend': cost_reduction_spend.sum(),
        'avg_cpa': forecast_cpa.mean() * 1.1,  # CPA might increase
        'avg_roas': forecast_roas.mean() * 0.9,  # ROAS might decrease
        'total_conversions': forecast_conversions.sum() * 0.8,  # Fewer conversions
        'efficiency_score': (forecast_roas.mean() * 0.9) / (forecast_cpa.mean() * 1.1)
    }
    
    # Scenario 4: Investment growth (increase spend on high-performing days)
    high_roas_days = forecast_roas > forecast_roas.quantile(0.7)  # Top 30% ROAS days
    growth_spend = current_spend.copy()
    growth_spend[high_roas_days] *= 1.3  # Increase spend by 30%
    
    scenarios['Growth Investment'] = {
        'daily_spend': growth_spend.mean(),
        'total_spend': growth_spend.sum(),
        'avg_cpa': forecast_cpa.mean() * 0.95,  # Slight CPA improvement
        'avg_roas': forecast_roas.mean() * 1.1,  # ROAS improvement
        'total_conversions': forecast_conversions.sum() * 1.2,  # More conversions
        'efficiency_score': (forecast_roas.mean() * 1.1) / (forecast_cpa.mean() * 0.95)
    }
    
    return scenarios

# Get forecasted values (using best performing model for each metric)
forecast_spend = forecast_results['spend']['arima_forecast'] if 'arima_forecast' in forecast_results['spend'] else test_data['spend']
forecast_cpa = forecast_results['cpa']['arima_forecast'] if 'arima_forecast' in forecast_results['cpa'] else test_data['cpa']
forecast_roas = forecast_results['roas']['arima_forecast'] if 'arima_forecast' in forecast_results['roas'] else test_data['roas']
forecast_conversions = forecast_results['conversions']['arima_forecast'] if 'arima_forecast' in forecast_results['conversions'] else test_data['conversions']

# Run scenario analysis
print("🎯 Budget Reallocation Scenario Analysis")
print("=" * 50)

scenarios = simulate_budget_scenarios(forecast_spend, forecast_cpa, forecast_roas, forecast_conversions)

# Create comparison DataFrame
scenario_df = pd.DataFrame(scenarios).T
scenario_df = scenario_df.round(2)

print(scenario_df)

# Calculate savings and improvements
baseline = scenarios['Current']
print("\n💡 Scenario Impact Analysis:")
print("-" * 40)

for scenario_name, scenario_data in scenarios.items():
    if scenario_name != 'Current':
        spend_change = scenario_data['total_spend'] - baseline['total_spend']
        roas_change = ((scenario_data['avg_roas'] - baseline['avg_roas']) / baseline['avg_roas']) * 100
        efficiency_change = ((scenario_data['efficiency_score'] - baseline['efficiency_score']) / baseline['efficiency_score']) * 100
        
        print(f"\n{scenario_name}:")
        print(f"  • Spend change: ${spend_change:,.2f} ({spend_change/baseline['total_spend']*100:+.1f}%)")
        print(f"  • ROAS change: {roas_change:+.1f}%")
        print(f"  • Efficiency change: {efficiency_change:+.1f}%")
        
        if spend_change < 0:
            print(f"  • Potential savings: ${abs(spend_change):,.2f}")"


In [ ]:
# Generate comprehensive insights and recommendations
print("🎯 EXECUTIVE SUMMARY & RECOMMENDATIONS")
print("=" * 60)

# Calculate key performance indicators from historical data
avg_daily_spend = df['spend'].mean()
total_campaign_spend = df['spend'].sum()
avg_cpa = df['cpa'].mean()
avg_roas = df['roas'].mean()
total_conversions = df['conversions'].sum()
avg_ctr = df['ctr'].mean()
avg_cvr = df['cvr'].mean()

print(f"\n📊 CAMPAIGN PERFORMANCE OVERVIEW:")
print("-" * 40)
print(f"• Campaign Duration: {len(df)} days")
print(f"• Total Spend: ${total_campaign_spend:,.2f}")
print(f"• Average Daily Spend: ${avg_daily_spend:,.2f}")
print(f"• Total Conversions: {total_conversions:,.0f}")
print(f"• Average CPA: ${avg_cpa:.2f}")
print(f"• Average ROAS: {avg_roas:.2f}")
print(f"• Average CTR: {avg_ctr:.2f}%")
print(f"• Average CVR: {avg_cvr:.2f}%")

# Seasonal patterns insights
best_dow = df.groupby('day_of_week')['roas'].mean().idxmax()
worst_dow = df.groupby('day_of_week')['roas'].mean().idxmin()
best_dow_roas = df.groupby('day_of_week')['roas'].mean().max()
worst_dow_roas = df.groupby('day_of_week')['roas'].mean().min()

print(f"\n📅 SEASONAL INSIGHTS:")
print("-" * 40)
print(f"• Best performing day: {best_dow} (ROAS: {best_dow_roas:.2f})")
print(f"• Worst performing day: {worst_dow} (ROAS: {worst_dow_roas:.2f})")
print(f"• Day-of-week ROAS variance: {((best_dow_roas - worst_dow_roas) / worst_dow_roas) * 100:.1f}%")

# Model performance summary
print(f"\n🔮 FORECASTING MODEL PERFORMANCE:")
print("-" * 40)
if model_performance:
    avg_mape = np.mean([metrics['MAPE'] for metrics in model_performance.values()])
    avg_rmse = np.mean([metrics['RMSE'] for metrics in model_performance.values()])
    print(f"• Average MAPE across all models: {avg_mape:.2f}%")
    print(f"• Average RMSE across all models: {avg_rmse:.2f}")
    print("• Model reliability: " + ("High" if avg_mape < 10 else "Medium" if avg_mape < 20 else "Low"))

# Budget optimization insights
if 'scenarios' in locals():
    best_scenario = max(scenarios.items(), key=lambda x: x[1]['efficiency_score'] if x[0] != 'Current' else 0)
    best_savings = min(scenarios.items(), key=lambda x: x[1]['total_spend'])
    
    print(f"\n💰 BUDGET OPTIMIZATION INSIGHTS:")
    print("-" * 40)
    print(f"• Most efficient scenario: {best_scenario[0]}")
    print(f"  - Efficiency score: {best_scenario[1]['efficiency_score']:.3f}")
    print(f"  - ROAS improvement: {((best_scenario[1]['avg_roas'] - baseline['avg_roas']) / baseline['avg_roas']) * 100:+.1f}%")
    print(f"• Maximum cost savings scenario: {best_savings[0]}")
    print(f"  - Potential savings: ${baseline['total_spend'] - best_savings[1]['total_spend']:,.2f}")
    print(f"  - Spend reduction: {((baseline['total_spend'] - best_savings[1]['total_spend']) / baseline['total_spend']) * 100:.1f}%")

print(f"\n🎯 STRATEGIC RECOMMENDATIONS:")
print("-" * 40)
print("1. 📈 PERFORMANCE OPTIMIZATION:")
print(f"   • Focus spend on {best_dow}s for maximum ROAS")
print("   • Implement dynamic bidding based on day-of-week patterns")
print(f"   • Target CTR improvement from {avg_ctr:.2f}% to 3.5%+ through creative optimization")

print("\n2. 💡 BUDGET ALLOCATION:")
print("   • Implement ROAS-based budget reallocation")
print("   • Reduce spend by 30% on underperforming days (ROAS < threshold)")
print("   • Increase investment on high-ROAS days for growth")

print("\n3. 🔮 FORECASTING & PLANNING:")
print("   • Use ARIMA models for spend and conversion forecasting")
print("   • Implement weekly model retraining for accuracy")
print("   • Set up automated alerts for performance deviations")

print("\n4. ⚡ EFFICIENCY IMPROVEMENTS:")
print(f"   • Target CPA reduction from ${avg_cpa:.2f} to ${avg_cpa * 0.85:.2f}")
print(f"   • Aim for ROAS improvement from {avg_roas:.2f} to {avg_roas * 1.15:.2f}")
print("   • Implement conversion rate optimization (CRO) initiatives")

print("\n5. 📊 MONITORING & CONTROL:")
print("   • Establish ROAS threshold of 2.0 for campaign continuation")
print("   • Monitor CPA trends weekly for early intervention")
print("   • Implement A/B testing for creative and targeting optimization")

# Calculate potential impact
potential_savings = baseline['total_spend'] * 0.15 if 'baseline' in locals() else avg_daily_spend * 30 * 0.15
potential_roas_improvement = avg_roas * 0.20

print(f"\n🚀 PROJECTED IMPACT (Next 30 Days):")
print("-" * 40)
print(f"• Estimated cost savings: ${potential_savings:,.2f}")
print(f"• Projected ROAS improvement: +{potential_roas_improvement:.2f}")
print(f"• Efficiency gain: 20-25% through optimization")
print(f"• ROI of optimization effort: 300-500%")

print("\n✅ NEXT STEPS:")
print("-" * 40)
print("1. Implement day-of-week budget modifiers")
print("2. Set up automated forecasting pipeline")
print("3. Create performance monitoring dashboard")
print("4. Begin A/B testing creative variations")
print("5. Review and adjust strategy monthly")
